## Installation de la bibliothèque :
 * pip install macronometrics-0.0.1.zip
 * vérifier le répertoire /donnees/colibri -> les fichiers small_db.csv et small_coeffs.csv doivent être présents.
 * vérifier le répertoire /codes_troll -> le fichier colibri.txt doit être présent.

In [1]:
import os
import pandas as pd
import sys
import numpy as np

In [2]:
import macronometrics.model as m

In [3]:
import macronometrics.tools as t
import macronometrics.tools_ts as ts

# Lecture du fichier d'équations définissant le modèle #

Un objet Model est instancié à partir du fichier d'équations. Pour plus d'informations sur les variables utilisées et les équations économétriques, voir : https://www.insee.fr/fr/statistiques/1380857 (dont Colibri est une simplification).

In [4]:
colibri=m.Model()
colibri.lexer("./codes_troll/colibriBF.txt")

# Chargement du modèle #

In [ ]:
colibri

Le modèle est composé d'équations, de variables (endogènes / exogènes / policy) et de coefficients (ayant vocation à être estimés).

In [ ]:
colibri.eq_obj_dict['tc_d41_s14s3']

# Construction de la fonction de résolution #

On vérifie que le modèle n'est pas encore construit, c'est à dire que les lignes de texte qui constituent les équations du modèle n'ont pas encore été interprétées pour constituer des objets sur lesquelles on va ensuite pouvoir travailler.

In [ ]:
colibri.is_built

On lance la fonction de construction du modèle, qui va utiliser l'algorithme de décomposition pour analyser la structure du système d'équations et la réduire. Un nom de fonction doit être donné : prendre le même nom que celui du modèle.

In [5]:
colibri.build_model("colibri")

The model has  54  equations.

54  endogenous variables declared.

The analysis of the model took 0.815 secondes.

The computation of the derivatives of the model took 0.003 secondes.

The block decomposition has 1 blocks.

The block decomposition took 0.823 seconds.

Building  the function took 0.008 seconds.



In [ ]:
colibri.is_built

A ce stade, deux types de fichiers ont été produits (dans le répertoire /modeles_python/) :
* Un fichier colibri.yaml permettant de vérifier les statuts des variables de chaque équation
* Un fichier colibri.py comprenant différentes fonctions nécessaires à la résolution du modèle

### Ouvrir le fichier colibri.yaml et comparer son contenu avec le fichier colibri.txt .

Chaque équation est désormais analysée en vue de la simulation.

In [ ]:
colibri.eq_obj_dict['tc_d41_s14s3']

# Chargement des données du modèle #

On va tout d'abord travailler avec une base où les cales ont été fixées à 0 (elles n'ont pas encore été calculées). Ces séries temporelles sont issues des comptes trimestriels publiés par l'Insee et sont rassemblées dans un fichier .csv .

In [6]:
datamodel=pd.read_csv("./donnees/colibri/small_db.csv",index_col=0,parse_dates=True)

On vérifie l'importation sur les dernières dates. Les cales ont un suffixe "_cale" dans le nom des variables.

In [ ]:
datamodel.tail()

# Chargement des coefficients #

On construit le dictionnaire des paramètres/coefficients du modèle à partir de leur importation. Plus tard, les coefficients pourront être initialisés sans valeur et celle ci sera estimée.

In [7]:
coeffmod = t.readcoeffs("./donnees/colibri/small_coeffs.csv")

In [8]:
eqtest = colibri.eq_obj_dict['tc_d11_d5']

In [9]:
eqtest

Equation : tc_d11_d5 
Texte : del(log(tc_d11_d5)) = c0d11d5'c +ar1d11d5'c*del(log(tc_d11_d5(-1))) +pc0d11d5'c*del(log(td_p3m_d5)) +dum1d11d5'c*(dum_1982q3-dum_1983q1) +c3d11dim5'c*log(tcho) +d11_d5_cale 
Numéro : 40 
Coefficients : ['ar1d11d5', 'c0d11d5', 'c3d11dim5', 'dum1d11d5', 'pc0d11d5'] 
Exogènes : ['dum_1982q3', 'dum_1983q1'] 
Policy : ['d11_d5_cale'] 
Endogènes contemporaines : ['tc_d11_d5', 'tcho', 'td_p3m_d5'] 
Endogènes retardées : ['tc_d11_d5', 'td_p3m_d5'] 

In [34]:
eqtest.print_tree()

define_eq
  deltaone
    log
      var	tc_d11_d5
  add
    add
      add
        add
          add
            coeff	c0d11d5
            mul
              coeff	ar1d11d5
              deltaone
                log
                  lag
                    var	tc_d11_d5
                    -1
          mul
            coeff	pc0d11d5
            deltaone
              log
                var	td_p3m_d5
        mul
          coeff	dum1d11d5
          par
            sub
              var	dum_1982q3
              var	dum_1983q1
      mul
        coeff	c3d11dim5
        log
          var	tcho
    var	d11_d5_cale



In [ ]:
eqtest.endo_name_list

In [ ]:
eqtest.endo_eq_dict

In [ ]:
eqtest.coeff_eq_dict

# Estimation d'une équation

In [10]:
import macronometrics.estimation as estim

Déclaration d'un objet Estim

In [11]:
test = estim.Estim(colibri.eq_obj_dict['tc_d11_d5'], colibri, datamodel)

Variables dans l'équation

['d11_d5_cale', 'dum_1982q3', 'dum_1983q1', 'tc_d11_d5', 'tcho', 'td_p3m_d5']
5 coefficient(s) à estimer

['ar1d11d5', 'c0d11d5', 'c3d11dim5', 'dum1d11d5', 'pc0d11d5']


In [12]:
test.coeff_eq_dict_loc

{'ar1d11d5': 0, 'c0d11d5': 1, 'c3d11dim5': 2, 'dum1d11d5': 3, 'pc0d11d5': 4}

In [13]:
test.var_eq_dict_loc

{'d11_d5_cale': 0,
 'dum_1982q3': 1,
 'dum_1983q1': 2,
 'tc_d11_d5': 3,
 'tcho': 4,
 'td_p3m_d5': 5}

Base de données locale contenant uniquement les séries nécessaires pour évaluer l'équation (faire attention à mettre la cale à zero)

In [14]:
test.data_eq

,d11_d5_cale,dum_1982q3,dum_1983q1,tc_d11_d5,tcho,td_p3m_d5
1978-01-01,0,0,0,1.853796,4.500000,0.346767
1978-04-01,0,0,0,1.908479,4.500000,0.355207
1978-07-01,0,0,0,1.973674,4.500000,0.365245
1978-10-01,0,0,0,2.028235,4.700000,0.373536
1979-01-01,0,0,0,2.073094,4.900000,0.383108
...,...,...,...,...,...,...
2005-10-01,0,0,0,7.474712,8.049891,1.077700
2006-01-01,0,0,0,7.528410,7.848790,1.080617
2006-04-01,0,0,0,7.583068,7.676880,1.083610
2006-07-01,0,0,0,7.638294,7.542621,1.086681


In [16]:
test.data_eq.iloc[8,:]

d11_d5_cale    0.000000
dum_1982q3     0.000000
dum_1983q1     0.000000
tc_d11_d5      2.356336
tcho           5.100000
td_p3m_d5      0.432740
Name: 1980-01-01 00:00:00, dtype: float64

In [17]:
dataset_eq = test.data_eq.to_numpy() # conversion en tableau numpy 

Création de la fonction d'estimation (somme des carrés des résidus)

In [18]:
test.create_estimfun_python()

In [19]:
print(test.fun_text)

	_res = 0
	for _t in range(_t_start,_t_stop):
		_res += (((log(_data[_t-0,3])- (log(_data[_t-1,3]))))-((((((_z[1])+((_z[0])*((log(_data[_t-1,3])- (log(_data[_t-2,3]))))))+((_z[4])*((log(_data[_t-0,5])- (log(_data[_t-1,5]))))))+((_z[3])*(((_data[_t-0,1])-(_data[_t-0,2])))))+((_z[2])*(log(_data[_t-0,4]))))+(_data[_t-0,0])))**2



In [22]:
def f_test(_z,_t_start,_t_stop,_data):
    _res = 0
    for _t in range(_t_start,_t_stop):
        _res += (((log(_data[_t-0,3])- (log(_data[_t-1,3]))))-((((((_z[1])+((_z[0])*((log(_data[_t-1,3])- (log(_data[_t-2,3]))))))+((_z[4])*((log(_data[_t-0,5])- (log(_data[_t-1,5]))))))+((_z[3])*(((_data[_t-0,1])-(_data[_t-0,2])))))+((_z[2])*(log(_data[_t-0,4]))))+(_data[_t-0,0])))**2
    return _res

Valeurs de test (documentation Grocer)

In [23]:
coeffmod['ar1d11d5']

0.33881479999999997

In [24]:
coeffmod['c0d11d5']

0.022298

In [25]:
coeffmod['c3d11dim5']

-0.008696200000000001

In [26]:
coeffmod['dum1d11d5']

-0.007490999999999999

In [27]:
coeffmod['pc0d11d5']

0.4251324000000001

In [29]:
from scipy.optimize import minimize

In [31]:
from math import log

In [35]:
minimize(f_test, [0,0,0,0,0], tol=1e-5, args=(8,113,dataset_eq))

      fun: 0.0013267899723080588
 hess_inv: array([[ 2.30976597e+02, -8.08581491e+00,  3.14883780e+00,
        -1.59628213e+00, -1.51913861e+02],
       [-8.08581491e+00,  2.57818005e+00, -1.11729959e+00,
        -2.85461564e-02, -8.56280484e+00],
       [ 3.14883780e+00, -1.11729959e+00,  4.86566761e-01,
         1.41886403e-02,  3.83485292e+00],
       [-1.59628213e+00, -2.85461564e-02,  1.41886403e-02,
         2.65372042e-01,  1.84073626e+00],
       [-1.51913861e+02, -8.56280484e+00,  3.83485292e+00,
         1.84073626e+00,  2.30904285e+02]])
      jac: array([-8.21644790e-07,  6.31553121e-09, -1.86264515e-09,  8.36735126e-09,
        7.62607669e-07])
  message: 'Optimization terminated successfully.'
     nfev: 175
      nit: 20
     njev: 25
   status: 0
  success: True
        x: array([ 0.33887722,  0.02200758, -0.00857382, -0.00747839,  0.42728935])

# Calage du modèle sur données historiques (1980Q3 - 2006Q4) #

La part d'inexpliqué du modèle est quantifiée par le calcul des valeurs des cales sur une période historique. Cette étape repose sur une étape d'inversion :
 * on rend endogènes les variables de type policy (les cales) qui étaient considérées initialement exogènes
 * on rend exogènes les variables endogènes historiques
 * on simule le modèle ainsi

Nom des endogènes associées à des cales dans le modèle :

In [ ]:
endo_cales = ["tc_d11_d5","tc_d41_s14s3","tc_d4z_s14s3","tc_d5_s14e3",
              "tc_emps_d7","td_p3a_d5","td_p3g_d5","td_p3m_d1","td_p3m_d5",
              "td_p51g_d5","td_p51m_d1","td_p51m_d5","td_p51t_d1","td_p51t_d5",
              "td_p523_d1","td_p523_d3","td_p6_d1","td_p6_d5","td_p7_d1",
              "td_p7_d5","td_pint_d5"]

On va procéder à une inversion du modèle pour calculer les valeurs historiques des cales, ce qui nécessite de définir un nouveau modèle tenant compte du changement de statut des variables.

In [ ]:
colibri_inv = colibri.copy()

In [ ]:
list_policy = colibri.name_policy_list.copy()

In [ ]:
list_policy

In [ ]:
for item in list_policy :
    t.changesym(colibri_inv,"endogenous",item)

In [ ]:
for item in endo_cales :
    t.changesym(colibri_inv,"exogenous",item)

On construit le nouveau modèle :

In [ ]:
colibri_inv.build_model('colibri_inv')

Le nouveau modèle a été construit (ainsi que les fonctions de résolution associées). On procède au calage :

In [ ]:
df_calage = t.simulate(datamodel,coeffmod,"1980Q3","2006Q4",'colibri_inv')

Les cales ont été calculées.

In [ ]:
df_calage[list_policy].tail()

Si tout se passe bien, si l'on calcule à nouveau les variables endogènes du modèle en prenant comme exogènes les cales ainsi calculées, on doit retrouver exactement les valeurs historiques initiales.

In [ ]:
df_verif = t.simulate(df_calage,coeffmod,"1980Q3","2006Q4",'colibri')

In [ ]:
variation_relative=abs(100*(df_verif/datamodel-1))
stat_rel=variation_relative[colibri.name_endo_list].describe()[colibri.name_endo_list]
stat_rel

In [ ]:
variation_absolue=df_verif-datamodel
stat_abs=variation_absolue.describe()[colibri.name_endo_list]
stat_abs

# Construction d'une base de données pour tester les fonctions d'extrapolations de séries.  #

Les cales et les exogènes sont prolongées à leur dernière valeur observée. Les tendances sont prolongées affines et les dummy trimestrielles sont prolongées.

In [ ]:
#Prolongement des cales à leur dernière valeur observée
liste_policy=[]
i=0
for item in colibri.name_policy_list:
    liste_policy.append([item,'constant',['2007Q1','2010Q4'],'last'])
    i+=1
    
db_extrapol_1=ts.extrapolate_series(df_calage,liste_policy)

In [ ]:
#Prolongement des exogènes à leur dernière valeur
liste_exo=[]
i=0
for item in colibri.name_exo_list:
    liste_exo.append([item,'constant',['2007Q1','2010Q4'],'last'])
    i+=1
db_extrapol_1=ts.extrapolate_series(db_extrapol_1,liste_exo)

In [ ]:
#Prolongement de la tendance
db_extrapol_1=ts.extrapolate_series(db_extrapol_1,[['time','affine',['2007Q1','2010Q4'],1,1]])

In [ ]:
#Prolongement des dummy trimestrielles
liste_dummy_trim=[['trim1','dummy_trim',['2007Q1','2010Q4'],[1,0,0,0]],
                 ['trim2','dummy_trim',['2007Q1','2010Q4'],[0,1,0,0]],
                 ['trim3','dummy_trim',['2007Q1','2010Q4'],[0,0,1,0]],
                 ['trim4','dummy_trim',['2007Q1','2010Q4'],[0,0,0,1]]]

db_extrapol_1=ts.extrapolate_series(db_extrapol_1,liste_dummy_trim)

In [ ]:
db_extrapol_1[colibri.name_exo_list].tail()

Le modèle peut ensuite être simulé sur cette nouvelle base de données prolongée, pour des dates qui n'étaient pas dans la base de données historiques initiale

In [ ]:
simul_1=t.simulate(db_extrapol_1,coeffmod,'2007Q1','2010Q4','colibri')

On peut faire de même en prolongant cette fois les exogènes en taux de croissance (en prévision d'une simulation d'un compte central)

In [ ]:
#Prolongement des cales à leur dernière valeur
liste_policy=[]
i=0
for item in colibri.name_policy_list:
    liste_policy.append([item,'constant',['2007Q1','2010Q4'],'last'])
    i+=1
    
db_extrapol_2=ts.extrapolate_series(df_calage,liste_policy)

In [ ]:
#Prolongement des exogènes à un taux de croissance de 3%
liste_exo=[]
i=0
for item in colibri.name_exo_list:
    liste_exo.append([item,'taux de croissance',['2007Q1','2010Q4'],3])
    i+=1
db_extrapol_2=ts.extrapolate_series(db_extrapol_2,liste_exo)

In [ ]:
#Prolongement de la tendance
db_extrapol_2=ts.extrapolate_series(db_extrapol_2,[['time','affine',['2007Q1','2010Q4'],1,1]])

In [ ]:
#Prolongement des dummy trimestrielles
liste_dummy_trim=[['trim1','dummy_trim',['2007Q1','2010Q4'],[1,0,0,0]],
                 ['trim2','dummy_trim',['2007Q1','2010Q4'],[0,1,0,0]],
                 ['trim3','dummy_trim',['2007Q1','2010Q4'],[0,0,1,0]],
                 ['trim4','dummy_trim',['2007Q1','2010Q4'],[0,0,0,1]]]

db_extrapol_2=ts.extrapolate_series(db_extrapol_2,liste_dummy_trim)

In [ ]:
db_extrapol_2[colibri.name_exo_list].tail()

In [ ]:
simul_2=t.simulate(db_extrapol_2,coeffmod,'2007Q1','2010Q4','colibri')

# On compare les deux simulations (en prévision d'une fonction permettant de présenter les résultats d'une variante) # 

In [ ]:
(comp_trim_abs,comp_ann_abs)=ts.compare_series('absolu',simul_1,simul_2,colibri.name_endo_list,'2007Q1','2010Q4')

In [ ]:
comp_ann_abs

In [ ]:
(comp_trim_rel,comp_ann_rel)=ts.compare_series('relatif',simul_1,simul_2,colibri.name_endo_list,'2007Q1','2010Q4')

In [ ]:
comp_ann_rel

# Simulation d'un choc permanent de demande mondiale de 1% à partir de 1990q1 #

In [ ]:
info_dict = {"variable" : ["demmon"] , "type" : "pctge" , "value" : 1}

In [ ]:
info = [info_dict]

In [ ]:
results=t.simul_shock('colibri',coeffmod,df_calage,start_date_shock='1990Q1',end_date_shock='2006Q4',start_date_sim='1985Q1',end_date_sim='2006Q4',info_shock=info)

In [ ]:
(choc_demmon_trim,choc_demmon_an)=ts.compare_series('relatif',df_calage,results,colibri.name_endo_list,'1985Q1','2006Q4')

In [ ]:
choc_demmon_an